## Diffusive routing applied to a basin above Falls Lake, NC using
   1. Crank Nicolson Hermite Interpolation derived by Prof.Ehab Meselhe, Tulane University and  
   coded by Md Nazmul Azim Beg, Postdoc of Prof.Ehab Meselhe.
   2. Fixed time step with internally/uniformly adjusting segment length inside of Fortran to    meet Courant condition, developed by Dong Ha Kim, NOAA's OWP, National Water Center
   3. Computation over entire time step and over entire network is done inside of Fortran while this Python routing framework v01 computes normal depth lookup table, irregular channel geometry lookup table (for this application, this table is prepared but not used inside Fotran), and Fortran-Python network traversal map.  The framework also pass channel geometry information, qlatral and boundary condition time series, and simulation time parameters to Fortran.
   
### Fortran source code: 
   1. nrtype.f90
   2. var_module.f90
   3. subtools.f90
   4. readXsection.f90
   5. uniformflowLT.f90
   6. mod_dx_tools.f90
   7. ef_q_y_calc.f90
   8. maindiff.f90
   
### Python py files:
   1. xsec_lookuptable_F.py
   2. routelink_adjustment.py
   3. waterdepth_lookuptable_F.py
   4. fortran_python_mapping.py
   5. set_boundary_F2.py
   6. qlateral_retrieval_F.py
   
### f2pyc wrapped library:
   dfcnhi_mt.cpython-37m-x86_64-linux-gnu.so
   
   
   

In [2]:
import sys
import os
import time
import numpy as np
#import globalvar as g
import dfcnhi_mt as df  

root=None

verbose = True
debuglevel = 0
showtiming = True
start_time= None

connections= None
supernetwork_data=None
network= None
ordered_reaches= None
seg_list_all= None
ncompall= None
fksegID_dsend= None
                   
dbsegID= None
dbfksegID= None

nel_g= None
xsec_attr_rch_g= None
nhincr_m_g= None
nhincr_f_g= None
ufqlt_m_g= None                       
ufhlt_m_g= None  
ufqlt_f_g= None  
ufhlt_f_g= None  
z_all= None

#pynw= None
#frnw_g= None
z_ar_g= None
bo_ar_g= None
traps_ar_g= None
tw_ar_g= None
twcc_ar_g= None
mann_ar_g = None
manncc_ar_g= None
so_ar_g=None
dx_ar_g= None 

mxncomp_g=None
nrch_g=None

pynw= None
frnw_g= None

output_path='./output2'

##-----------------------------------------------------------------------------------
#    Set channel geometry data, xsec_tables, normal depth tables, and 
#    Fortran to Python Mapping tables
#
##-----------------------------------------------------------------------------------
def setting_hydTables_Fr_PyMap():
    
    global root
    global start_time 
    
    global connections
    global supernetwork_data
    global network
    global ordered_reaches
    global seg_list_all
    global ncompall
    global fksegID_dsend
        
    global dbsegID
    global dbfksegID

    global nel_g
    global xsec_attr_rch_g
    global nhincr_m_g
    global nhincr_f_g
    global ufqlt_m_g                       
    global ufhlt_m_g 
    global ufqlt_f_g 
    global ufhlt_f_g 
    
    global z_all
    global z_ar_g
    global bo_ar_g
    global traps_ar_g
    global tw_ar_g
    global twcc_ar_g
    global mann_ar_g
    global manncc_ar_g
    global so_ar_g
    global dx_ar_g
    
    global mxncomp_g
    global nrch_g
    
    global pynw
    global frnw_g


    from sys import platform
    if platform == "linux" or platform == "linux2":
        pass
    elif platform == "darwin":
        pass
    elif platform == "win32":
        print('The parallel version of compute_nhd_routing.py will not execute as currently')
        print('written due to the lack of a fork() capability in the windows OS.')
        print('For parallel execution, please us a *nix OS.')
        print('\nexiting...')
        sys.exit()
        # Some useful references:
        # https://stackoverflow.com/questions/985281/what-is-the-closest-thing-windows-has-to-fork/985525#985525
        # https://stackoverflow.com/questions/8220108/how-do-i-check-the-operating-system-in-python
        # https://stackoverflow.com/questions/6596617/python-multiprocess-diff-between-windows-and-linux
    ENV_IS_CL = False
    if ENV_IS_CL: root = '/content/t-route/'
    elif not ENV_IS_CL: 
        sys.setrecursionlimit(4000)
    #     root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
        root = os.path.dirname(os.path.dirname(os.path.abspath('')))
        sys.path.append(os.path.join(root, r'src', r'python_framework'))
        sys.path.append(os.path.join(root, r'src', r'python_routing')) #by Dong Ha
        sys.path.append(os.path.join(root, r'src', r'evaluation',r'_restclient')) # for usgs data retrieval
        sys.path.append(os.path.join(root, r'src', r'evaluation',r'nwis_client')) # for usgs data retrieval
        fortran_source_dir = os.path.join(root, r'src', r'fortran_routing', r'diffCNHI_pylink')
        sys.path.append(fortran_source_dir)

    ## F2PY fortran source codes of diffusive routing using Crank-Nicolson Hermite Interpolation 
    # mc_f2py= True
    # if mc_f2py:
    #     try:
    #         import subprocess

    #         f2py_call = []
    #         f2py_call.append(r'f2py3')
    #         f2py_call.append(r'-c')
    #         f2py_call.append(r'nrtype.f90')
    #         f2py_call.append(r'var_module.f90')
    #         f2py_call.append(r'subtools.f90')
    #         f2py_call.append(r'readXsection.f90')
    #         f2py_call.append(r'uniformflowLT.f90')
    #         f2py_call.append(r'calculateDT.f90')
    #         f2py_call.append(r'ef_q_y_calc.f90')
    #         f2py_call.append(r'-m')
    #         f2py_call.append(r'dfcnhi')
    #         subprocess.run(f2py_call, cwd=r'../fortran_routing/diffCNHI_pylink',
    #                        stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    #     except Exception as e:
    #         print(e)

    #** network and reach utilities **
    import nhd_network_utilities as nnu
    import nhd_reach_utilities as nru
    import numpy as np
    import statistics as stat
    import math
    #import pixiedust
    import xsec_lookuptable_F as xseclt
    import routelink_adjustment as rladj
    import waterdepth_lookuptable_F as wdlt
    import fortran_python_mapping as fpm

    #global connections
    #global geo_input_folder

    test_folder = os.path.join(root, r'test')
    geo_input_folder = os.path.join(test_folder, r'input', r'geo', r'Channels')
    # mainly for retrieving qlateral
    custom_input_folder = os.path.join(root, r'test', r'input', 'yaml') 
    custom_input_file = "florence_933020089_dt300.yaml"
    custom_input_file_path=os.path.join(custom_input_folder,custom_input_file)  


    #TODO: Make these commandline args
    #supernetwork= 'Pocono_TEST3_3R1J'
    #supernetwork='Pocono_TEST2'  # 29 reaches for MC parity test
    supernetwork='Florence_933020089'
    """##NHD Subset (Brazos/Lower Colorado)"""
    #     supernetwork = 'Brazos_LowerColorado_ge5'
    """##NHD CONUS order 5 and greater"""
    # supernetwork = 'CONUS_ge5'
    """These are large -- be careful"""                          
    #     supernetwork = 'Mainstems_CONUS'
    # supernetwork = 'CONUS_FULL_RES_v20'
    # supernetwork = 'CONUS_Named_Streams' #create a subset of the full resolution by reading the GNIS field
    # supernetwork = 'CONUS_Named_combined' #process the Named streams through the Full-Res paths to join the many hanging reaches

    if verbose: print('creating supernetwork connections set')
    if showtiming: start_time = time.time()
    #STEP 1
    supernetwork_data, supernetwork_values = nnu.set_networks(
        supernetwork = supernetwork
        , geo_input_folder = geo_input_folder
        , verbose = False
        # , verbose = verbose
        , debuglevel = debuglevel
        )
    if verbose: print('supernetwork connections set complete')
    if showtiming: print("... in %s seconds." % (time.time() - start_time))

    #STEP 2
    if showtiming: start_time = time.time()
    if verbose: print('organizing connections into networks and reaches ...')
    networks = nru.compose_reaches(
        supernetwork_values
        , verbose = False
        # , verbose = verbose
        , debuglevel = debuglevel
        , showtiming = showtiming
        )
    if verbose: print('reach organization complete')
    if showtiming: print("... in %s seconds." % (time.time() - start_time))


    #STEP 3: Route NHD streamflow
    if showtiming: start_time = time.time()
    executiontype = 'serial' # 'parallel'

    if verbose: print('executing serial computation on ordered reaches ...')
    connections = supernetwork_values[0]


    #----------------------------------------------------------
    # x-sec attribute lookup table variables

    #----------------------------------------------------------    
    nxsecpt_g =8 #the number of (x,y) points for x-sec geometry
    nel_g=21 # the number of incrementally increasing water depth lines
    timesdepth_g=5.0  # multiplier to bankfull depth to cover potentially largest water depth
    #-------------------------------------------------------------
    #  uniform flow discharge-normal depth lookup table variables

    #-------------------------------------------------------------     
    nhincr_m_g=5  # the number of depth increments of main channel
    nhincr_f_g=10 # the number of depth increments of floodplain
    #-------------------------------------------------------------
    #  downstream boundary parameters

    #-------------------------------------------------------------     
    #dbsegID= 4185661  # downstream boundary segment ID for supernetwork= 'Pocono_TEST3_3R1J'
    #dbsegID= 4186169  # downstream boundary segment ID for supernetwork='Pocono_TEST2' 
    dbsegID= 933020089  # downstream boundary segment ID for supernetwork='Florence_933020089' 
    #dbseg_usgsID='02087182' #usgs site ID corresponding to dbsegID for downstream boundary condition
    
    #dbsegID: downstream boundary segment ID -> make a fake segment
    dbfksegID= str(dbsegID) + str(2)
    dbfksegID= int(dbfksegID) 

    iter=0
    #for terminal_segment, network in networks.items():     
    if showtiming: 
        network_start_time = time.time()
    iter=iter+1         
    #print(f"terminal_segment:{terminal_segment} network:{network}")
    #import pdb; pdb.set_trace()
    #toseg=connections[terminal_segment]['data'][supernetwork_data['downstream_col']]
    #if toseg != 0 or terminal_segment==dbsegID:

    terminal_segment= dbsegID
    network= networks[terminal_segment]
    #print(f"terminal_segment{terminal_segment} toseg:{toseg}")    
    print(f"terminal_segment iter-> {iter}")   

    # ** x-sec attribute lookup table variables **    
    # nel: the number of incrementally increasing water depth lines
    xsec_attr_all={connection:{'elev':np.zeros(nel_g)
                        ,'convey':np.zeros(nel_g)
                        ,'topwd':np.zeros(nel_g)
                        ,'uniflow':np.zeros(nel_g)}
                           for connection in connections} 
    # ** uniform flow discharge-normal depth lookup table variables **
    # nhincr_m_g: the number of depth increments of main channel
    # nhincr_f_g: the number of depth increments of floodplain
    uflt={connection:{'q_m':np.zeros(nhincr_m_g)
                      ,'h_m':np.zeros(nhincr_m_g)
                      ,'q_f':np.zeros(nhincr_f_g)
                      ,'h_f':np.zeros(nhincr_f_g)}
                        for connection in connections} 
    # ** channel attribute adjustment **
    z_all={connection:{'adj.alt':np.zeros(1)}
                        for connection in connections} 
    # dictionary all keyed by head_segment key
    ordered_reaches = {}
    last_segment_reach={}
    lsegs_usrch={}
    ncompall={}
    fksegID_dsend={}
    seg_list_all={}
    hseg_list=[]
    rchlen_all={}

    for head_segment, reach in network['reaches'].items():
        if reach['seqorder'] not in ordered_reaches:
            ordered_reaches.update({reach['seqorder']:[]}) 
        # Ordered_reaches contains network dictionary values according to seqorder (= 
        # the number of juctions downstream).
        ordered_reaches[reach['seqorder']].append([head_segment, reach])
        last_segment_reach[head_segment]= reach['reach_tail']
        hseg_list.append(head_segment)

    # Count the total number of segments for each reach in computation
    # ** Note that routing models use node configuration where one segment has
    # ** has upper and lower nodes and each segment shares common nodes at either 
    # ** direction. So, the number of nodes of a given reaach becomes equiv. to
    # ** the number of segments + 1.
    tnseg=0
    nrch_g=0 # the number of reaches
    mxncomp_g=0 # max. number of segments within a reach
    for x in range(network['maximum_reach_seqorder'],-1,-1):  
        for head_segment, reach in ordered_reaches[x]:                        
            seg_list0=list(reach['segments_list'])
            seg_list=seg_list0[::-1] #to reversed order
            # check if there is a downstream boundary segment in the given reach.
            if seg_list0.count(dbsegID)>0:
                # identify where dsbdID is located along the given reach, and count 
                # segments upward from and including the dsbdID segment.                    
                idx= seg_list0.index(dbsegID)
                tnseg= tnseg+len(seg_list)-idx
                ncomp= len(seg_list)-idx+1 
            else:
                #the number of segments of a given reach 
                tnseg= tnseg+ len(seg_list)
                ncomp= len(seg_list)+1
            ncompall.update({head_segment:ncomp})
            if ncomp>mxncomp_g:
                mxncomp_g= ncomp
            nrch_g= nrch_g +1 # counts the number of reaches 
            # store reach length of each link for evalution later
            for seg in range(0,ncomp-1):                    
                segID= seg_list[seg-1]                
                rchlen=connections[segID]['data'][supernetwork_data['length_col']]
                rchlen_all.update({segID:rchlen})
    #---------------------------------------------------------------------------------------
    #                            Step 0-2    

    #  Update key dictionary variables
    #---------------------------------------------------------------------------------------
    # Add one more fake segment after downstream terminal segment in a reach
    # by "terminal segmentID"+"2". This is to account for one more node in
    # Fortran node configuration, which is equivalent to Python segment 
    # configuration.
            fksegID= seg_list[ncomp-2]
            fksegID= str(fksegID) + str(2)
            fksegID=int(fksegID)
            fksegID_dsend.update({head_segment:fksegID})                     

            xsec_attr_all.update({fksegID:{'elev':np.zeros(nel_g)
                                        ,'convey':np.zeros(nel_g)
                                        ,'topwd':np.zeros(nel_g)
                                        ,'uniflow':np.zeros(nel_g)}})

            uflt.update({fksegID:{'q_m':np.zeros(nhincr_m_g)
                                  ,'h_m':np.zeros(nhincr_m_g)
                                  ,'q_f':np.zeros(nhincr_f_g)
                                  ,'h_f':np.zeros(nhincr_f_g)}})

            seg_list.append(fksegID_dsend[head_segment])

            seg_list_all.update({head_segment:seg_list})

            z_all.update({fksegID:{'adj.alt':np.zeros(1)}}) 

    cel_av_g=np.zeros(nrch_g) # holds reach-averaged celerities
    #import pdb; pdb.set_trace()
    #--------------------------------------------------------------------------------------
    #                                 Step 0-3           

    #    Adjust altitude so that altitude of the last sement of a reach is equal to that 
    #    of the first segment of its downstream reach right after their common junction.
    #--------------------------------------------------------------------------------------
    rladj.adj_alt1(        
                    connections= connections 
                    , supernetwork_data= supernetwork_data 
                    , network= network
                    , ordered_reaches= ordered_reaches
                    , seg_list_all= seg_list_all
                    , ncompall= ncompall 
                    , dbfksegID= dbfksegID
                    , z_all= z_all
                    ) 
    #--------------------------------------------------------------------------------------
    #                                 Step 0-4           

    #     Make Fortran-Python channel network mapping variables.
    #--------------------------------------------------------------------------------------   
    pynw={}
    frj=-1
    for x in range(network['maximum_reach_seqorder'],-1,-1):       
        for head_segment, reach in ordered_reaches[x]:
            frj= frj+1
            pynw[frj]=head_segment        
    frnw_g=np.zeros((nrch_g,8), dtype=int)
    
    fpm.fpmap1(
            connections= connections 
            , supernetwork_data= supernetwork_data 
            , network= network
            , ordered_reaches= ordered_reaches
            , seg_list_all= seg_list_all
            , ncompall= ncompall
            , nrch_g= nrch_g
            , dbfksegID= dbfksegID
            , z_all= z_all
            , pynw= pynw
            , frnw_g= frnw_g
            )  
    
    pynwf=np.zeros((nrch_g,2), dtype=int)
    frj=-1
    for x in range(network['maximum_reach_seqorder'],-1,-1):       
        for head_segment, reach in ordered_reaches[x]:
            frj= frj+1
            pynwf[frj,0]= frj+1 # +1 is added to accommodate Fortran indexing
            pynwf[frj,1]=head_segment
            
    np.savetxt("./output2/frnw_ar.txt", frnw_g, fmt='%10i')
    np.savetxt("./output2/pynw_ar.txt", pynwf, fmt='%20i')
    #---------------------------------------------------------------------------------
    #                              Step 0-5

    #                  Prepare channel geometry data           
    #---------------------------------------------------------------------------------    
    z_ar_g=np.zeros((mxncomp_g, nrch_g))
    bo_ar_g=np.zeros((mxncomp_g, nrch_g))
    traps_ar_g=np.zeros((mxncomp_g, nrch_g))
    tw_ar_g=np.zeros((mxncomp_g, nrch_g))
    twcc_ar_g=np.zeros((mxncomp_g, nrch_g))
    mann_ar_g=np.zeros((mxncomp_g, nrch_g))
    manncc_ar_g=np.zeros((mxncomp_g, nrch_g))
    so_ar_g=np.zeros((mxncomp_g, nrch_g))
    dx_ar_g=np.zeros((mxncomp_g, nrch_g))
    frj=-1
    for x in range(network['maximum_reach_seqorder'],-1,-1):   
        for head_segment, reach in ordered_reaches[x]:                  
            seg_list= seg_list_all[head_segment] 
            ncomp=ncompall[head_segment]              
            frj=frj+1
            for seg in range(0,ncomp):                    
                if seg==ncomp-1:
                    segID= seg_list[seg-1]
                else:
                    segID= seg_list[seg]                               

                bo_ar_g[seg, frj]=connections[segID]['data'][supernetwork_data['bottomwidth_col']]                
                traps=connections[segID]['data'][supernetwork_data['ChSlp_col']]
                traps_ar_g[seg, frj]=1.0/traps  # s in 1 over s for channel side slope               
                tw_ar_g[seg, frj]=connections[segID]['data'][supernetwork_data['topwidth_col']]
                twcc_ar_g[seg, frj]=connections[segID]['data'][supernetwork_data['topwidthcc_col']]
                mann_ar_g[seg, frj]=connections[segID]['data'][supernetwork_data['manningn_col']]
                manncc_ar_g[seg, frj]=connections[segID]['data'][supernetwork_data['manningncc_col']]
                so_ar_g[seg, frj]=connections[segID]['data'][supernetwork_data['slope_col']]
                dx_ar_g[seg, frj]=connections[segID]['data'][supernetwork_data['length_col']]
                segID1= seg_list[seg]  
                z_ar_g[seg, frj]= z_all[segID1]['adj.alt'][0]    
                
             
    # saved txt file has row for segment and colunum for frj
    np.savetxt("./output2/z_ar.txt", z_ar_g, fmt='%15.5f')
    np.savetxt("./output2/bo_ar.txt", bo_ar_g, fmt='%15.5f')
    np.savetxt("./output2/traps_ar.txt", traps_ar_g, fmt='%15.5f')
    np.savetxt("./output2/tw_ar.txt", tw_ar_g, fmt='%15.5f')
    np.savetxt("./output2/twcc_ar.txt", twcc_ar_g, fmt='%15.5f')
    np.savetxt("./output2/mann_ar.txt", mann_ar_g, fmt='%15.5f')
    np.savetxt("./output2/manncc_ar.txt", manncc_ar_g, fmt='%15.5f')
    np.savetxt("./output2/so_ar.txt", so_ar_g, fmt='%15.6f') 
    np.savetxt("./output2/dx_ar.txt", dx_ar_g, fmt='%15.5f')
    
    
    #---------------------------------------------------------------------------------------
    #                                 Step 0-6

    #  Make channel x-sec attribute lookup tables for each node (p.132,rm4)
    #  ** tzeq_flag_g= 0: use an approximation method based on finite rectangles or triangle 
    #                       for any regular or irregular x-sec
    #                1: use related equations only for trapezoidal main & rectangular 
    #                        floodplains
    #  ** Output: 
    #            xsec_attr_all
    #              ** this variable also stores uniform flow values corresponding to elev.
    #----------------------------------------------------------------------------------------
    xseclt.xsecTR_ltable(        
                        connections= connections
                        , supernetwork_data= supernetwork_data
                        , network= network
                        , ordered_reaches= ordered_reaches
                        , seg_list_all= seg_list_all
                        , ncompall= ncompall
                        , z_all= z_all
                        , nxsecpt_g= nxsecpt_g
                        , nel_g= nel_g
                        , timesdepth_g= timesdepth_g
                        , tzeq_flag_g= 1
                        , xsec_attr_all= xsec_attr_all
                        )

    xsec_attr_rch_g=np.zeros((mxncomp_g, nrch_g, nel_g, 4))
    frj=-1
    #with open(os.path.join(output_path,"Python_xsec_attr_rch_g"),'a') as xarch:
    for x in range(network['maximum_reach_seqorder'],-1,-1):       
        for head_segment, reach in ordered_reaches[x]:           
            seg_list= seg_list_all[head_segment] 
            ncomp=ncompall[head_segment]   
            frj=frj+1
            for seg in range(0,ncomp):                  
                segID= seg_list[seg]
                for i1 in range(0,nel_g):
                    xsec_attr_rch_g[seg, frj, i1,0]= xsec_attr_all[segID]['elev'][i1]
                    xsec_attr_rch_g[seg, frj, i1,1]= xsec_attr_all[segID]['convey'][i1]
                    xsec_attr_rch_g[seg, frj, i1,2]= xsec_attr_all[segID]['topwd'][i1]
                    xsec_attr_rch_g[seg, frj, i1,3]= xsec_attr_all[segID]['uniflow'][i1]
                    #if i1==0:
                    #    print(f"xsec_attr_rch_g seg:{seg} frj:{frj} segID:{segID}")
                    #xarch.write("%s %s %s %s %s %s %s\n" %\
                    #           (seg, frj, segID,\
                    #            xsec_attr_rch_g[seg,frj,i1,0],\
                    #            xsec_attr_rch_g[seg,frj,i1,1],\
                    #            xsec_attr_rch_g[seg,frj,i1,2],\
                    #            xsec_attr_rch_g[seg,frj,i1,3],\
                    #            ))
    #---------------------------------------------------------------------------------
    #                              Step 0-7

    #  Create uniform flow-normal depth lookup tables using channel geometry equations
    #  ONLY for trapezoidal main and rectangular floodplains (p.132,rm4)            
    #---------------------------------------------------------------------------------
    ufqlt_m_g= np.zeros((mxncomp_g,nrch_g,nhincr_m_g))                        
    ufhlt_m_g= np.zeros((mxncomp_g,nrch_g,nhincr_m_g))
    ufqlt_f_g= np.zeros((mxncomp_g,nrch_g,nhincr_f_g))
    ufhlt_f_g= np.zeros((mxncomp_g,nrch_g,nhincr_f_g))
    
    wdlt.ufTR_ltable_frnw(        
                    connections= connections 
                    , supernetwork_data= supernetwork_data
                    , network= network
                    , ordered_reaches= ordered_reaches
                    , seg_list_all= seg_list_all
                    , ncompall= ncompall
                    , mxncomp_g= mxncomp_g
                    , nrch_g= nrch_g
                    , frnw_g= frnw_g
                    , timesdepth_g= timesdepth_g
                    , nhincr_m_g= nhincr_m_g
                    , nhincr_f_g= nhincr_f_g
                    , ufqlt_m_g= ufqlt_m_g
                    , ufhlt_m_g= ufhlt_m_g
                    , ufqlt_f_g= ufqlt_f_g
                    , ufhlt_f_g= ufhlt_f_g
                    , bo_ar_g=bo_ar_g
                    , traps_ar_g= traps_ar_g
                    , tw_ar_g=tw_ar_g
                    , twcc_ar_g=twcc_ar_g
                    , mann_ar_g= mann_ar_g
                    , manncc_ar_g= manncc_ar_g
                    , so_ar_g= so_ar_g      
                    )       
    
#    frj=-1
    #with open(os.path.join(output_path,"Python_uftable_f"),'a') as uftb:
#    for x in range(network['maximum_reach_seqorder'],-1,-1): 
#        for head_segment, reach in ordered_reaches[x]:                  
#            seg_list= seg_list_all[head_segment] 
#            ncomp=ncompall[head_segment]
#            frj=frj+1
#            for seg in range(0,ncomp):                  
#                segID= seg_list[seg]
#                for i1 in range(0,nhincr_m_g):
#                    ufqlt_m_g[seg, frj, i1]=uflt[segID]['q_m'][i1]
#                    ufhlt_m_g[seg, frj, i1]=uflt[segID]['h_m'][i1]
#                for i1 in range(0,nhincr_f_g):
#                    ufqlt_f_g[seg, frj, i1]=uflt[segID]['q_f'][i1] 
#                    ufhlt_f_g[seg, frj, i1]=uflt[segID]['h_f'][i1]
                    #uftb.write("%s %s %s %s %s\n" %\
                    #        (i1, frj, segID,\
                    #        ufqlt_f_g[seg,frj,i1], ufhlt_f_g[seg,frj,i1]))                        
    #print(f"uflt inside:{uflt}")
  
                            
    if verbose: print(f'{terminal_segment} completed')
    if showtiming: print("... in %s seconds." % (time.time() - network_start_time))

    #if verbose: print('ordered reach computation complete')
    #if showtiming: print("... in %s seconds." % (time.time() - start_time))

        
       
    
def set_run_diff_routing():
  
    import set_boundarydata_F2 as setbd
    import qlateral_retrieval_F as qlrtv
    
    # mainly for retrieving qlateral
    custom_input_folder = os.path.join(root, r'test', r'input', 'yaml') 
    custom_input_file = "florence_933020089_dt300.yaml"
    custom_input_file_path=os.path.join(custom_input_folder,custom_input_file)  
    output_path='./output2'
    
    #------------------------------------------------------
    #        diffusive CNHI model time variables
    #------------------------------------------------------  
    dtini_g= 300.0   # initial time interval in [sec]
    t0_g= 0.0        # simulation starting time in [hr]
    tfin_g= 18.0      # simulation ending time in [hr] 
    cfl_g= 0.99      # max. allowable Courant number 
    dt_ql_g= 3600.0  # instead of dtinput #time interval of qlateral input data from wrf-hydro [sec]
    dt_ub_g= dt_ql_g # time interval of input data for upstream boundary condition (= headbasin seg lat.flow)  [sec]
    dt_db_g=  900.0  # time interval of input data for downstream boundary condition  [sec]
    saveinterval_ev_g= dtini_g # for evaluation later [sec]
    usgssDT='2018-08-01'       # start date of USGS retrieval tool
    usgseDT='2018-08-02'       # end date of USGS retrieval tool
    dbseg_usgsID='02087182'    # usgs site ID corresponding to dbsegID for downstream boundary condition
    
    ntss_ev_g= int((tfin_g - t0_g)*3600.0/saveinterval_ev_g)+1
    #ydaq_ev = {connection:{'elev':np.zeros(ntss_ev_g)
    #                        ,'q':np.zeros(ntss_ev_g)
    #                        ,'celerity':np.zeros(ntss_ev_g)
    #                        ,'diffusivity':np.zeros(ntss_ev_g)}
    #                        for connection in connections}
    # ** lateral inflows **
    # qlateral time variables
    nts_ql_g= (tfin_g - t0_g)*3600.0/dt_ql_g+1 # based on time of [sec]
    nts_ql_g= int(nts_ql_g)
    qlatral = {connection:{'qlat':np.zeros(nts_ql_g)
                        } for connection in connections}            
    # ** upstream boundary condition ** 
    # For Florence case, headbasin segment discharge=0 m^3/sec.
    nts_ub_g= nts_ql_g 
    ubcond={connection:{'elev':np.zeros(nts_ub_g)
                        , 'discharge':np.zeros(nts_ub_g)
                        } for (connection, cv) in connections.items()
                                                 if cv['upstreams']=={0}}
    # ** downstream boundary condition **
    # For Florence case, measured Fall lake elevation is used.
    nts_db_g= (tfin_g - t0_g)*3600.0/dt_db_g+1 # based on time of [sec]
    nts_db_g=int(nts_db_g)
    dbcond={dbfksegID:{'elev':np.zeros(nts_db_g)
                      , 'discharge':np.zeros(nts_db_g)} } 
    
    #** add one more fake segment to each varialble **
    for x in range(network['maximum_reach_seqorder'],-1,-1):  
        for head_segment, reach in ordered_reaches[x]:      
            fksegID=fksegID_dsend[head_segment]   
            
            #ydaq_ev.update({fksegID:{'elev':np.zeros(ntss_ev_g)
            #                        ,'q':np.zeros(ntss_ev_g)
            #                        ,'celerity':np.zeros(ntss_ev_g)
            #                        ,'diffusivity':np.zeros(ntss_ev_g)}})

            qlatral.update({fksegID:{'qlat':np.zeros(nts_ql_g)}})

   
    #-----------------------------------------------------------------------------------------
    #                                       Step 1

    # Set boundary data such as discharge, water depth, lateral flow by either data import or 
    # artificial equations.       
    # ** the unit of qlat that is eventually used in the routing SHOULD BE [m^2/sec].
    # ** qlat of fake segment of the last segment of a reach is always ZERO.
    #-----------------------------------------------------------------------------------------            
    #setbd.set_bdrydata(        
    #                connections= connections  
    #                , supernetwork_data= supernetwork_data
    #                , network= network
    #                , ordered_reaches= ordered_reaches
    #                , seg_list_all= seg_list_all
    #                , ncompall= ncompall 
    #                , fksegID_dsend= fksegID_dsend
    #                , dbfksegID= dbfksegID
    #                , nts_ql_g= nts_ql_g
    #                , nts_ub_g= nts_ub_g
    #                , nts_db_g= nts_db_g                    
    #                , bdrydata= "artf_data1"
    #                , z_all= z_all
    #                , ubcond= ubcond
    #                , dbcond= dbcond 
    #                , qlatral= qlatral
    #                )  
    qlrtv.retrieve_qlatral(        
                            connections= connections 
                            , supernetwork_data= supernetwork_data
                            , network= network
                            , ordered_reaches= ordered_reaches
                            , seg_list_all= seg_list_all
                            , ncompall= ncompall
                            , fksegID_dsend= fksegID_dsend
                            , nts_ql_g= nts_ql_g # ntsi= ntsi                            
                            , custom_input_folder = custom_input_folder
                            , custom_input_file= custom_input_file
                            , custom_input_file_path= custom_input_file_path
                            , output_path= output_path
                            , qlatral= qlatral
                            )    
    # ** qlateral flow mapping
    df.var.qlat_g=np.zeros((nts_ql_g, mxncomp_g, nrch_g))    
    frj= -1
    with open(os.path.join(output_path,"qlat_ar.txt"),'a') as pyql:
        for x in range(network['maximum_reach_seqorder'],-1,-1):   
            for head_segment, reach in ordered_reaches[x]:                  
                seg_list= seg_list_all[head_segment] 
                ncomp=ncompall[head_segment]              
                frj= frj+1     
                for seg in range(0,ncomp):                               
                    segID= seg_list[seg]
                    for tsi in range (0,nts_ql_g):
                        df.var.qlat_g[tsi,seg,frj]= qlatral[segID]['qlat'][tsi] 
                        t_min= t0_g*60.0 + dt_ql_g/60.0*float(tsi)                        
                        pyql.write("%s %s %s %s\n" %\
                                (frj+1, seg+1, t_min,df.var.qlat_g[tsi,seg,frj]))  
                                # <- +1 is added to frj for accommodating Fortran indexing.
                
    setbd.set_bdrydata_usgs1(        
                    connections= connections  
                    , supernetwork_data= supernetwork_data
                    , network= network
                    , ordered_reaches= ordered_reaches
                    , seg_list_all= seg_list_all
                    , ncompall= ncompall 
                    , fksegID_dsend= fksegID_dsend
                    , dbfksegID= dbfksegID
                    , dbseg_usgsID= dbseg_usgsID
                    , nts_ql_g= nts_ql_g
                    , nts_ub_g= nts_ub_g
                    , nts_db_g= nts_db_g
                    , ubcond= ubcond
                    , dbcond= dbcond  
                    , usgssDT= usgssDT
                    , usgseDT= usgseDT
                    , qlatral= qlatral
                    , output_path= output_path
                    )       
                                
    # ** upstream boundary mapping
    df.var.ubcd_g=np.zeros((nts_ub_g, nrch_g))  
    nl_ubcd_ar_g=0
    frj=-1
    with open(os.path.join(output_path,"ubcd_ar.txt"),'a') as ub:
        for x in range(network['maximum_reach_seqorder'],-1,-1):       
            for head_segment, reach in ordered_reaches[x]:                  
                frj=frj+1
                # ** headwater reach or upstream reaches
                if reach['upstream_reaches']=={0}:
                    nrow=0
                    for tsi in range(0,nts_ub_g):
                        df.var.ubcd_g[tsi, frj]= ubcond[head_segment]['discharge'][tsi]
                        nl_ubcd_ar_g= nl_ubcd_ar_g+1
                        # write to file
                        t_min= t0_g*60.0 + dt_ub_g/60.0*float(tsi)                        
                        nrow=nrow+1 #<- for fortran row index
                        ub.write("%s %s %s %s\n" %\
                            (frj+1, nrow, t_min, df.var.ubcd_g[tsi, frj]))  
    print(f"nl_ubcd_ar_g:{nl_ubcd_ar_g}")                    
    
    
    # ** downstream boundary mapping
    df.var.dbcd_g=np.zeros(nts_db_g)
    with open(os.path.join(output_path,"dbcd_ar.txt"),'a') as db:
        for tsi in range(0,nts_db_g):
            df.var.dbcd_g[tsi]= dbcond[dbfksegID]['elev'][tsi]
            # write to file
            t_min= t0_g*60.0 + dt_db_g/60.0*float(tsi)                        
            db.write("%s %s\n" %\
                    (t_min, df.var.dbcd_g[tsi]))  
    
    #import pdb; pdb.set_trace()
    #------------------------------------------------------------------------------------
    #                                   Step 2

    #    Intial condition of q (p.130,rm4)
    #------------------------------------------------------------------------------------
#    dfrt.icond_q(          
#                connections= connections
#                , supernetwork_data= supernetwork_data
#                , network= network
#                , ordered_reaches= ordered_reaches
#                , fksegID_dsend= fksegID_dsend
#                , seg_list_all= seg_list_all
#                , ncompall= ncompall
#                , z_all= z_all
#                , ubcond=ubcond 
#                , ydaq= ydaq
#                , qlatral= qlatral
#                ) 
#    df.var.q_g=np.zeros((ntss, mxncomp_g, nrch_g)) 
#    frj= -1
#    for x in range(network['maximum_reach_seqorder'],-1,-1):   
#        for head_segment, reach in ordered_reaches[x]:                  
#            seg_list= seg_list_all[head_segment] 
#            ncomp=ncompall[head_segment]              
#            frj= frj+1     
#            for seg in range(0,ncomp):                               
#                segID= seg_list[seg]            
#                df.var.q_g[0,seg,frj]= ydaq[segID]['q'][0] 
 
    #import pdb; pdb.set_trace()
    #-------------------------------------------------------------------------------------
    #                                     Step 3

    #       Run diffusive routing based on Crank-Nicholson & Hermite Interpolation
    #         
    #-------------------------------------------------------------------------------------    
    
    # ** set variables in Fortran var module
    df.var.nel_g=nel_g   
    #df.var.nxsecpt_g = nxsecpt_g  <- used only locally
    
    df.var.dtini_g =dtini_g 
    df.var.t0_g = t0_g        # simulation starting time in hr
    df.var.tfin_g = tfin_g    # simulation ending time in hr
    df.var.cfl_g = cfl_g      # max. allowable Courant number 
    #df.var.saveinterval_g = saveinterval_g  # output publish time interval in sec  # <- not used
    df.var.saveinterval_ev_g = saveinterval_ev_g # <- not used
    #xcs_g, ycs_g <- used only locally 
    #xsec_attr_seg_g(:,:) <- used only locally 
    
    #df.var.xsec_attr_rch_g=np.zeros((mxncomp_g, nrch_g, nel_g, 4)) <- this table set to not be used.
    #df.var.xsec_attr_rch_g= xsec_attr_rch_g
    
    #timesDepth_g <- used only locally
    tzeq_flag_g=1
    df.var.tzeq_flag_g= tzeq_flag_g
    #z_g, bo_g, traps_g, tw_g, twcc_g, So_g, mann_g, manncc_g  <- used only locally 
    
    dx_mn_g=min([connections[segID]['data'][supernetwork_data['length_col']] for segID in connections]) 
    df.var.dx_mn_g=dx_mn_g         # minimum length value among all segments of a network
    
    theta_g=1.0
    df.var.theta_g= theta_g
    
    y_opt_g = 1  # 1 for normal depth(kinematic); 2 for dept of diffusive wave.
    df.var.y_opt_g= y_opt_g  
    
    df.var.mxncomp_g= mxncomp_g
    df.var.nrch_g= nrch_g
    
    df.var.nts_ql_g= nts_ql_g   
    df.var.nts_ub_g= nts_ub_g
    df.var.nts_db_g= nts_db_g 
    df.var.ntss_ev_g= ntss_ev_g
    
    df.var.nhincr_m_g= nhincr_m_g
    df.var.nhincr_f_g= nhincr_f_g       
    
    df.var.dt_ql_g= dt_ql_g
    df.var.dt_ub_g= dt_ub_g
    df.var.dt_db_g= dt_db_g 
    
    df.var.ufhlt_m_g= np.zeros((mxncomp_g,nrch_g,nhincr_m_g))    
    df.var.ufqlt_m_g= np.zeros((mxncomp_g,nrch_g,nhincr_m_g))                        
    df.var.ufhlt_f_g= np.zeros((mxncomp_g,nrch_g,nhincr_f_g))  
    df.var.ufqlt_f_g= np.zeros((mxncomp_g,nrch_g,nhincr_f_g))
    
    df.var.ufhlt_m_g= ufhlt_m_g 
    df.var.ufqlt_m_g= ufqlt_m_g
    df.var.ufhlt_f_g= ufhlt_f_g
    df.var.ufqlt_f_g= ufqlt_f_g

    df.var.frnw_g= np.zeros((nrch_g,8), dtype=int)
    df.var.frnw_g= frnw_g    

    df.var.z_ar_g= np.zeros((mxncomp_g, nrch_g))
    df.var.bo_ar_g= np.zeros((mxncomp_g, nrch_g))
    df.var.traps_ar_g= np.zeros((mxncomp_g, nrch_g))
    df.var.tw_ar_g= np.zeros((mxncomp_g, nrch_g))
    df.var.twcc_ar_g= np.zeros((mxncomp_g, nrch_g))
    df.var.mann_ar_g= np.zeros((mxncomp_g, nrch_g))
    df.var.manncc_ar_g= np.zeros((mxncomp_g, nrch_g))
    df.var.so_ar_g= np.zeros((mxncomp_g, nrch_g))
    df.var.dx_ar_g= np.zeros((mxncomp_g, nrch_g))
    
    df.var.z_ar_g= z_ar_g
    df.var.bo_ar_g= bo_ar_g
    df.var.traps_ar_g= traps_ar_g
    df.var.tw_ar_g= tw_ar_g
    df.var.twcc_ar_g= twcc_ar_g
    df.var.mann_ar_g= mann_ar_g
    df.var.manncc_ar_g= manncc_ar_g
    df.var.so_ar_g= so_ar_g   
    df.var.dx_ar_g= dx_ar_g
    
    df.var.q_ev_g= np.zeros((ntss_ev_g, mxncomp_g, nrch_g))
    df.var.elv_ev_g= np.zeros((ntss_ev_g, mxncomp_g, nrch_g))   

    #celty_g, diffty_g, q_g, elv_g, qpx_g<- used only locally 
    #cel_av_g<- used only locally 
    #qlatj_g<- used only locally 
    #eei_g, ffi_g, exi_g, fxi_g<- used only locally 
    #df.var.diagflg_time=0.5 #* for only print [min]. Temporary
    
    df.var.so_llm_g= 0.0005  # minimally allowed channel bottom slope

    
    df.mdiff.routenw()   



 # Run this function
setting_hydTables_Fr_PyMap()  
set_run_diff_routing()

print('ordered reach computation complete')
print("... in %s seconds." % (time.time() - start_time))

#print(f"main_frnw_g:{frnw_g}")
#print(f"main_pynw:{pynw}")




creating supernetwork connections set
supernetwork connections set complete
... in 0.22204995155334473 seconds.
organizing connections into networks and reaches ...
reach organization complete
... in 0.0054857730865478516 seconds.
executing serial computation on ordered reaches ...
terminal_segment iter-> 1
933020089 completed
... in 2.842141628265381 seconds.
creating qlateral array ...
qlateral array complete
... in 0.7913336753845215 seconds.
nl_ubcd_ar_g:7068
ordered reach computation complete
... in 5.860745668411255 seconds.
